<a href="https://colab.research.google.com/github/pyaidev/air-ticket-forecast-Ml/blob/main/avia_chiptalar_bashorati.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'aviachipta-narxini-bashorat-qilish:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F36503%2F3619833%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240410%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240410T082627Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D24d6b8f0b605abe765204fe4d272d888e7f2cd809f23452fd6f88fcf2e7d1323f12bd44f9ca39dd17f5bed1c57eb96f565e71267453067452439fc7ee63435e3ab955424c3da9665403b0850e046953fb09e31a42974b31c2cc3076ecab0c4c2e3142aad4165762d6ff1fccfb7b7a933eefb0738bd0d75346589de0ad2ca80f56c63174799f86ed358399b345a41199b105b07bdf4bcc2363058dc390775a580c99983ebffde5a39005813d066a86ce843a972a670e3165e37d824ad279e2ec8a00f5232be4d1e005cc741caf0206352d89160296a29c2dad4b85f993dd9b843eb0ebce5451859b9589ec471678e00f943c707eac9d4edf0f32b672f4e82fc7f'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Импорт Библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import  OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from catboost import CatBoostRegressor

%matplotlib inline

### Чтение CSV-файлов¶

In [ ]:
df = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/train_data.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/test_data.csv", index_col=0)
df_submission = pd.read_csv("/kaggle/input/aviachipta-narxini-bashorat-qilish/sample_solution.csv", index_col=0)
df.head(10)

### Формирование обучающих и тестовых наборов

In [ ]:
X_train = df.drop('price', axis=1)
y = df['price']
X_test = df_test

### Pipeline



In [ ]:
X_num = X_train[['duration', 'days_left']]
X_cat = X_train.drop(columns=['flight', 'stops', 'class', 'duration', 'days_left'])

In [ ]:
class StopsEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # This method is typically used for any necessary setup, but we don't need it here.
        return self

    def transform(self, X, y=None):
        # Map categories to numerical values
        X['stops'] = X['stops'].map({'zero': 0, 'one': 1, 'two_or_more': 2})
        return X

In [ ]:
class ClassEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # This method is typically used for any necessary setup, but we don't need it here.
        return self

    def transform(self, X, y=None):
        # Map categories to numerical values
        X['class'] = X['class'].map({'Economy': 0, 'Business': 1})
        return X

In [ ]:
class FlightEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # This method is typically used for any necessary setup, but we don't need it here.
        return self

    def transform(self, X, y=None):
        # Map categories to numerical values
        X[['flight_country', 'flight_number']]=X["flight"].str.split("-", expand=True)
        X['flight_number']=X['flight_number'].astype('int64')

        encoder = OrdinalEncoder()
        col_ordinal_encoder = X[['flight_country']]
        X[['flight_country']] = encoder.fit_transform(col_ordinal_encoder)

        X.drop(['flight'], axis=1, inplace=True)
        return X

In [ ]:
class OheEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        # This method is typically used for any necessary setup, but we don't need it here.
        return self

    def transform(self, X, y=None):
        # Map categories to numerical values

        one_hot_encoder = ['airline', 'source_city', 'departure_time', 'arrival_time','destination_city']
        X = pd.get_dummies(X, columns=one_hot_encoder)

        # X['duration'] = X['duration']
        # X['days_left'] = X['days_left']
        return X

In [ ]:
# num_pipeline = Pipeline([
#     ('std_scaler', StandardScaler())
# ])

cat_pipeline = Pipeline([
    ('ohe', OheEncoder())
])

stops_pipeline = Pipeline([
    ('stops_encoder', StopsEncoder())
])

class_pipeline = Pipeline([
    ('class_encoder', ClassEncoder())
])

flight_pipeline = Pipeline([
    ('flight_encoder', FlightEncoder())
])

In [ ]:
# Full pipeline

# num_attribs = list(X_num)
cat_attribs = list(X_cat)
stops_attribs = ['stops']
class_attribs = ['class']
flight_attribs = ['flight']

full_pipeline = ColumnTransformer(transformers=[
    # ('num', num_pipeline, num_attribs),
    ('cat', cat_pipeline, cat_attribs),
    ('stops', stops_pipeline, stops_attribs),
    ('class', class_pipeline, class_attribs),
    ('flight', flight_pipeline, flight_attribs)], remainder='passthrough'
)

In [ ]:
X_train_prepared = full_pipeline.fit_transform(X_train)
X_test_prepared = full_pipeline.transform(X_test)

### Machine Learning
* #  Random Forest

In [ ]:
RF_model = RandomForestRegressor()
RF_model.fit(X_train_prepared, y)
y_RF_predicted = RF_model.predict(X_test_prepared)

In [ ]:
submission_rf = df_submission.copy()
submission_rf

In [ ]:
submission_rf['price'] = y_RF_predicted
submission_rf

In [ ]:
submission_rf.to_csv("submission_rf.csv")

### XGBoost

In [ ]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=13)
xgb_model.fit(X_train_prepared, y)
y_XGB_predicted = xgb_model.predict(X_test_prepared)

In [ ]:
submission_xgb = df_submission.copy()
submission_xgb

In [ ]:
submission_xgb['price'] = y_XGB_predicted
submission_xgb

In [ ]:
submission_xgb.to_csv("submission_xgb.csv")

### CatBoost

In [ ]:
cb_model = CatBoostRegressor(iterations=1000,  # Number of boosting iterations
                                   learning_rate=0.1,  # Learning rate
                                   depth=8,  # Depth of the decision tree
                                   loss_function='RMSE',  # Loss function for regression
                                   random_seed=42,  # Random seed for reproducibility
                                   verbose=200)  # Print progress every 200 iterations
cb_model.fit(X_train_prepared, y)
y_CB_predicted = cb_model.predict(X_test_prepared)

In [ ]:
submission_cb = df_submission.copy()
submission_cb

In [ ]:
submission_cb['price'] = y_CB_predicted
submission_cb

In [ ]:
submission_cb.to_csv("submission_cb.csv")